In [ ]:
import requests
import csv
import pandas as pd
import json
import numpy as np
from datetime import date, timedelta, time, datetime
from time import sleep

# The date issue
It is only possible to get a maximum of 200 data points per request from the API. It is hence necessary to find the data for every 200 hours between January 1st 2017 and January 1st 2025.

The following code calculates the data and hour for every period lasting slightly less 200 hours between January 1st 2017 and January 1st 2025. We must convert this list back to a datetime.date object as the API request does not take specific hours. In the ideal world we would have looped through a list of list consisting of exactly 200 hour intervals e.g. [2017-01-01-00, 2017-01-09-08]. But now, we convert the list to normal dates again and then make a list of list with each object as a time interval of around 200 hrs fe. [2017-01-01, 2017-01-09].

 We can later loop through this list of lists but will have double hours in the data. These need to be removed during the data cleaning process. 

In [ ]:
def daterange(start_date, end_date):
    #for n in range(0, int((end_date - start_date).total_seconds()/3600), 200):
    for n in range(0, int((end_date - start_date).days*24), 176):
        yield start_date + timedelta(hours = n)
         
#create empty list to store dates
datelist = []
#define start and end date for list of dates
start_day = date(2017, 1, 1)
start_time = time(00)
start_dt = datetime.combine(start_day, start_time)
end_day = date(2025, 1, 1)
end_time = time(00)
end_dt = datetime.combine(end_day, end_time)
#append dates to list
for hrs in daterange(start_dt, end_dt):
    hrs = hrs.strftime('%Y-%m-%d-%H')
    datelist.append(hrs)

len(datelist) 

In [ ]:
dates_datelist = []
for i in datelist:
    date_format = '%Y-%m-%d-%H'
    i = datetime.strptime(i, date_format).strftime('%Y-%m-%d')
    dates_datelist.append(i)

datelists = [item for item in dates_datelist for i in range(2)]
datelists = datelists[1:]
datelists.append("2025-01-02")
print(datelists[-1])
#remove_double_end = datelists.pop(-1)
out = [datelists[i: i+2] for i in range(0, len(datelists), 2)]
print(len(out))

# Loop to get all the offshore data

In [ ]:
url = "https://api.ned.nl/v1/utilizations?page=1&itemsPerPage=500&order%5Bvalidfrom%5D=desc"
Dataframes_all = []
for i, j in out:
    print(f"Processing: {i} to {j}")
    headers_ij = {
    'X-AUTH-TOKEN': 'ff760e9d4bacda20d72d33c6b06f8350310594eae88a05ba2268140dfe0469d8',
    'accept': 'application/ld+json'}
    params_ij = {'point': 14, 'type': 17, 'granularity': 5, 'granularitytimezone': 1, 'classification': 2, 'activity': 1,
 'validfrom[strictly_before]': j, 'validfrom[after]': i}
    response_ij = requests.get(url, headers=headers_ij, params=params_ij, allow_redirects=False)
    sleep(3)
    print(response_ij.status_code)
    text_ij = response_ij.text
    json_text_ij = json.loads(text_ij)

    values_ij = [list(n.values()) for n in json_text_ij['hydra:member']]
    headers_ij = list(json_text_ij['hydra:member'][0].keys())

    # Create DataFrame
    data_ij = pd.DataFrame(values_ij, columns=headers_ij)

    # Append directly, without extra list nesting
    Dataframes_all.append(data_ij)
    print(f"Dataframes list length: {len(Dataframes)}")

In [ ]:
final_result = pd.concat(Dataframes_all, ignore_index=True)
print(final_result.shape)
final_result.isnull().sum()
#final_result.to_csv("All_Days_WOS.csv")

# Loop to get all the onshore data

In [1]:
url = "https://api.ned.nl/v1/utilizations?page=1&itemsPerPage=500&order%5Bvalidfrom%5D=desc"
Dataframes_onsh = []
for i, j in out:
    print(f"Processing: {i} to {j}")
    headers_ij = {
    'X-AUTH-TOKEN': 'ff760e9d4bacda20d72d33c6b06f8350310594eae88a05ba2268140dfe0469d8',
    'accept': 'application/ld+json'}
    params_ij = {'point': 0, 'type': 1, 'granularity': 5, 'granularitytimezone': 1, 'classification': 2, 'activity': 1,
 'validfrom[strictly_before]': j, 'validfrom[after]': i}
    response_ij = requests.get(url, headers=headers_ij, params=params_ij, allow_redirects=False)
    sleep(3)
    print(response_ij.status_code)
    text_ij = response_ij.text
    json_text_ij = json.loads(text_ij)

    values_ij = [list(n.values()) for n in json_text_ij['hydra:member']]
    headers_ij = list(json_text_ij['hydra:member'][0].keys())

    # Create DataFrame
    data_ij = pd.DataFrame(values_ij, columns=headers_ij)

    # Append directly, without extra list nesting
    Dataframes_onsh.append(data_ij)
    print(f"Dataframes list length: {len(Dataframes_onsh)}")

NameError: name 'out' is not defined

In [ ]:
final_result_onsh = pd.concat(Dataframes_onsh, ignore_index=True)
print(final_result_onsh.shape)
final_result_onsh.isnull().sum()
#final_result.to_csv("All_Days_WOS.csv")